In [1]:
from model import *
import random
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from model import cifar_mobilenet, cifar_discriminator_model, cifar_decoder, cifar_pseudo, vgg16
from model1 import BottomModelForCifar10, TopModelForCifar10
from vfl import VFLNN, Client, Server
import torch.backends.cudnn as cudnn


In [3]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    id = 'cuda:'+ '0'
    device = torch.device(id)
    torch.cuda.set_device(id)
    # cudnn.benchmark = True
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [2]:
import copy


cinic_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
            ])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True, transform=cinic_transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train = False, transform=cinic_transform, download=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers = 4, pin_memory = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers = 4, pin_memory = True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
target_iterator = iter(train_dataloader)
target_data, target_labels = next(target_iterator)
target_data, target_labels = target_data.to(device), target_labels.to(device)

In [6]:
discriminator = cifar_discriminator_model(3, 2).to(device)
d_ouput = discriminator(target_data)
print(d_ouput.size())

torch.Size([64, 4096])
